In [2]:
import pandas as pd
import numpy as np
import ast
import math
from pyvis.network import Network
import sys
sys.path.append('../')
from modules import utils, constants

#### The test df

In [3]:
model_name = 'dueling_dqn_per'
seed = 42
steps = int(10e7)

In [4]:
test_df = pd.read_csv(f'../test_dfs/{model_name}_risk_factor_{seed}_{steps}.csv')
test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,16.0,0.385286,1.0,1.0,"['leukopenia', 'anti_dsdna_antibody', 'low_c3'...",0.0,1.0
1,1.0,11.0,0.609372,1.0,1.0,"['leukopenia', 'anti_dsdna_antibody', 'pericar...",0.0,1.0
2,2.0,11.0,0.624625,1.0,1.0,"['leukopenia', 'anti_dsdna_antibody', 'low_c3'...",0.0,1.0
3,3.0,20.0,0.167750,1.0,1.0,"['leukopenia', 'anti_dsdna_antibody', 'pericar...",0.0,1.0
4,4.0,8.0,0.718047,1.0,1.0,"['leukopenia', 'anti_dsdna_antibody', 'low_c3'...",0.0,1.0


In [5]:
utils.test(test_df.y_actual, test_df.y_pred)[0]

97.57142857142857

In [6]:
test_df[(test_df.y_pred==0) & (test_df.episode_length!=2)].episode_length.mean()

11.741108025915642

#### The other pertinent dfs

In [7]:
testing_df = pd.read_csv('../data/test_set_constant.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [8]:
training_df = pd.read_csv('../data/train_set_basic.csv')
training_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


#### Possibly useful functions

In [9]:
def generate_tuple_dict(df):
    frequency_dict = {}
    for traj in df.trajectory:
        if traj in frequency_dict.keys():
            frequency_dict[traj] += 1
        else:
            frequency_dict[traj] = 1
    overall_tup_dict = {}
    for key, value in frequency_dict.items():
        new_key = ast.literal_eval(key)
        for tup in zip(new_key, new_key[1:]):
            if tup in overall_tup_dict.keys():
                overall_tup_dict[tup] += value
            else:
                overall_tup_dict[tup] = value
    return overall_tup_dict

In [10]:
def create_sankey_df(df):
    overall_tup_dict = generate_tuple_dict(df)
    sankey_df = pd.DataFrame()
    sankey_df['source'] = [i[0] for i in overall_tup_dict.keys()]
    sankey_df['target'] = [i[1] for i in overall_tup_dict.keys()]
    sankey_df['value'] = list(overall_tup_dict.values())
    sankey_df['link_type'] = sankey_df['target'].apply(lambda i: 'terminal' if i in ['No lupus', 'Lupus', 'Inconclusive diagnosis'] else 'non_terminal')
    return sankey_df

In [11]:
def draw_pyvis_network(pathways_df, pathway_type='network'): #second type is trajectory 
    pathways_df['type'] = 'directed'
    start_node = pathways_df.iloc[0]['source']
    all_nodes = list(set(pathways_df.source.unique().tolist() + pathways_df.target.unique().tolist()))
    terminal_nodes = list(set(pathways_df[pathways_df.link_type=='terminal'].target))
    non_terminal_nodes = [i for i in all_nodes if i not in terminal_nodes and i!=start_node]
    
    #change get_net to something else since it satnds for Game of Thrones
    got_net = Network(notebook=True, height='600px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value, edge_thresh in zip(pathways_df.source, pathways_df.target, pathways_df.value, pathways_df.edge_threshold):
        if pathway_type =='network':
            if value > threshold:
                if math.isnan(edge_thresh):
                    got_net.add_edge(src, target, value=value, color='red')
                else:
                    got_net.add_edge(src, target, value=value, color='red', label=str(int(edge_thresh)))
            else:
                got_net.add_edge(src, target, value=value, color='blue')
        elif pathway_type =='trajectory': #It's just one trajectory
            if isinstance(edge_thresh, str):
                got_net.add_edge(src, target, value=value, color='blue', label=edge_thresh)
            else:
                got_net.add_edge(src, target, value=value, color='blue', label=str(int(edge_thresh)))
        else:
            print('Unknown pathway type')
    return got_net

In [12]:
def get_threshold_value(df, source): #source e.g. 'ana'
    threshold_values = df[source].unique().tolist()
    if len(threshold_values) == 1:
        return int(threshold_values[0])
    else:
        return str(threshold_values)

In [13]:
def draw_one_trajectory(pred_df, trajectory): #pred_df = pred_no_lupus par exemple
    trajectory_df = pred_df[pred_df.trajectory == trajectory]
    trajectory_testing_df = testing_df.loc[trajectory_df.index]
    trajectory_pathways_df = create_sankey_df(trajectory_df)
    trajectory_pathways_df['edge_threshold'] = [get_threshold_value(trajectory_testing_df, source) for source in trajectory_pathways_df.source]
    trajectory_net = draw_pyvis_network(trajectory_pathways_df, 'trajectory')   
    return trajectory_net

#### No lupus diagnosis

##### shortest pathway

In [14]:
pred_no_lupus = test_df[test_df['y_pred']==0]
shortest_no_lupus_path = pred_no_lupus[pred_no_lupus.episode_length == pred_no_lupus.episode_length.min()].trajectory.unique()[0]
shortest_no_lupus_path

"['leukopenia', 'anti_dsdna_antibody', 'low_c3', 'low_c4', 'pericardial_effusion', 'ana', 'No lupus']"

In [15]:
shortest_no_lupus_path_net = draw_one_trajectory(pred_no_lupus, shortest_no_lupus_path)
shortest_no_lupus_path_net.show('Example.html')

Example.html


##### commonest pathway 

In [20]:
commonest_no_lupus_path = pred_no_lupus.trajectory.value_counts().index[0]
commonest_no_lupus_path_net = draw_one_trajectory(pred_no_lupus, commonest_no_lupus_path)
commonest_no_lupus_path_net.show('Example.html')

Example.html


In [34]:
len(pred_no_lupus[pred_no_lupus.trajectory == commonest_no_lupus_path])

3954

##### second commonest pathway

In [21]:
commonest_no_lupus_path2 = pred_no_lupus.trajectory.value_counts().index[1]
commonest_no_lupus_path2_net = draw_one_trajectory(pred_no_lupus, commonest_no_lupus_path2)
commonest_no_lupus_path2_net.show('Example.html')

Example.html


In [35]:
len(pred_no_lupus[pred_no_lupus.trajectory == commonest_no_lupus_path2])

562

##### longest pathways = 37

In [24]:
pred_no_lupus[pred_no_lupus.episode_length == pred_no_lupus.episode_length.max()].trajectory.value_counts()[:5]

['ana', 'anti_dsdna_antibody', 'joint_involvement', 'low_c3', 'pericardial_effusion', 'pleural_effusion', 'proteinuria', 'delirium', 'thrombocytopenia', 'leukopenia', 'lupus_anti_coagulant', 'oral_ulcers', 'seizure', 'anti_β2gp1_antibodies', 'low_c4', 'psychosis', 'anti_cardioliphin_antibodies', 'fever', 'non_scarring_alopecia', 'anti_smith_antibody', 'cutaneous_lupus', 'auto_immune_hemolysis', 'acute_pericarditis', 'No lupus']    29
['ana', 'anti_dsdna_antibody', 'joint_involvement', 'low_c3', 'pleural_effusion', 'lupus_anti_coagulant', 'pericardial_effusion', 'proteinuria', 'thrombocytopenia', 'delirium', 'psychosis', 'oral_ulcers', 'anti_smith_antibody', 'seizure', 'anti_β2gp1_antibodies', 'leukopenia', 'acute_pericarditis', 'fever', 'anti_cardioliphin_antibodies', 'cutaneous_lupus', 'non_scarring_alopecia', 'auto_immune_hemolysis', 'low_c4', 'No lupus']    27
['ana', 'anti_dsdna_antibody', 'joint_involvement', 'pleural_effusion', 'proteinuria', 'low_c3', 'pericardial_effusion', 'lu

In [29]:
longest_no_lupus_path1 = pred_no_lupus[pred_no_lupus.episode_length==pred_no_lupus.episode_length.max()].trajectory.value_counts().index[0]
longest_no_lupus_path2 = pred_no_lupus[pred_no_lupus.episode_length==pred_no_lupus.episode_length.max()].trajectory.value_counts().index[1]
longest_no_lupus_path3 = pred_no_lupus[pred_no_lupus.episode_length==pred_no_lupus.episode_length.max()].trajectory.value_counts().index[2]
longest_no_lupus_path1

"['ana', 'anti_dsdna_antibody', 'joint_involvement', 'low_c3', 'pericardial_effusion', 'pleural_effusion', 'proteinuria', 'delirium', 'thrombocytopenia', 'leukopenia', 'lupus_anti_coagulant', 'oral_ulcers', 'seizure', 'anti_β2gp1_antibodies', 'low_c4', 'psychosis', 'anti_cardioliphin_antibodies', 'fever', 'non_scarring_alopecia', 'anti_smith_antibody', 'cutaneous_lupus', 'auto_immune_hemolysis', 'acute_pericarditis', 'No lupus']"

In [30]:
longest_no_lupus_path1_net = draw_one_trajectory(pred_no_lupus, longest_no_lupus_path1)
longest_no_lupus_path1_net.show('Example.html')

Example.html


In [36]:
len(pred_no_lupus[pred_no_lupus.trajectory == longest_no_lupus_path1])

29

In [31]:
longest_no_lupus_path2_net = draw_one_trajectory(pred_no_lupus, longest_no_lupus_path2)
longest_no_lupus_path2_net.show('Example.html')

Example.html


In [37]:
len(pred_no_lupus[pred_no_lupus.trajectory == longest_no_lupus_path2])

27

In [32]:
longest_no_lupus_path3_net = draw_one_trajectory(pred_no_lupus, longest_no_lupus_path3)
longest_no_lupus_path3_net.show('Example.html')

Example.html


In [38]:
len(pred_no_lupus[pred_no_lupus.trajectory == longest_no_lupus_path3])

25

In [2]:
samp_pathway = str(['ana', 'joint_involvement', 'anti_dsdna_antibody', 'low_c3', 'anti_smith_antibody', 'fever', 'psychosis', 'non_scarring_alopecia', 'pleural_effusion', 'cutaneous_lupus', 'seizure', 'leukopenia', 'low_c4', 'pericardial_effusion', 'No lupus'])
samp_pathway

"['ana', 'joint_involvement', 'anti_dsdna_antibody', 'low_c3', 'anti_smith_antibody', 'fever', 'psychosis', 'non_scarring_alopecia', 'pleural_effusion', 'cutaneous_lupus', 'seizure', 'leukopenia', 'low_c4', 'pericardial_effusion', 'No lupus']"

In [17]:
a = pred_no_lupus[(pred_no_lupus.trajectory==samp_pathway)&(pred_no_lupus.y_actual != pred_no_lupus.y_pred)]
draw_one_trajectory(a, samp_pathway).show('Example.html')

Example.html


In [24]:
len(test_df[test_df.trajectory==samp_pathway])

195